In [1]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation, load_step_function

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [2]:
#hg_model_hub_name = "alisawuffles/roberta-large-wanli"
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
nli_model = NLIModel(hg_model_hub_name, device='cuda')

In [24]:
# MAIN GOAL


melt_plan = """
    IF your goal is to melt mercury THEN
        move to kitchen,
        pick up thermometer,
        get metal pot,
        get mercury,
        focus on substance in metal pot,
        freeze mercury,
        heat mercury on blast furnace,
        focus on substance in metal pot
"""

# get metal pot
subplan_a = """
    IF your goal is to get metal pot CONSIDERING you are in the kitchen AND you see a cupboard THEN
        open cupboard,
        pick up metal pot
"""


subplan_b = """
    IF your goal is to get mercury CONSIDERING you are not in workshop AND you have metal pot in your inventory THEN
        move to workshop,
        pour glass cup into metal pot in inventory
"""

subplan_c = """
    IF your goal is to freeze mercury CONSIDERING you have a metal pot with mercury in your inventory THEN
        move to workshop,
        open freezer,
        move metal pot to freezer,
        wait,
        wait,
        use thermometer on metal pot,
        focus on substance in metal pot,
        pick up metal pot
"""

# heat water on stove
subplan_d = """
    IF your goal is to heat mercury on blast furnace CONSIDERING you have metal pot with mercury in your inventory THEN
        move to foundry,
        open blast furnace,
        move metal pot to blast furnace,
        activate blast furnace,
        use thermometer on metal pot,
        focus on substance in metal pot,
        wait,
        use thermometer on metal pot
"""

all_plans = [melt_plan, subplan_a, subplan_b, subplan_c, subplan_d]

pl = PlanLibrary()
#pl.load_plans_from_strings(all_plans)  # load plans from strings above
#pl.load_plans_from_file("plans_navigation.txt")  # load plans from file
print(pl.plans.keys())

In [25]:
lengths = [len(plan) for key, plan in pl.plans.items()]
sum(lengths)

In [70]:
env = ScienceWorldEnv("", "", envStepLimit=100)

#root_event = 'use chemistry to create green paint'

task = 'melt'
env.load(task, 0)
#randVariationIdx = env.getRandomVariationTest()
randVariationIdx = 24
env.load(task, randVariationIdx)

goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])
print(current_state.look)
goal

In [71]:
step_function = load_step_function(env, goal=goal)
agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.reward)

In [6]:
last_state.look